In [ ]:
# -*- coding: utf-8 -*-

from jaqs.data import DataApi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

def get_fundlist(api, invest_type, invest_style):
    df, msg = api.query(
                view="lb.mfInfo",
                fields="invest_type,invest_style,status",
                filter="",
                data_format='pandas')
    
    #print(df, msg)
    
    df = df[(df['invest_type'] == invest_type) 
            & (df['invest_style'] == invest_style) 
            & (df['status'] == 101001000) 
            & (df['name'].apply(lambda s: 'A' not in s))]
    
    return df

def get_fundnav(api, symbol, start_date, end_date):
    df, msg = api.query(
                      view="lb.mfNav",
                      fields="price_date, nav_adjusted",
                      filter="start_pdate=" + str(start_date) + "&end_pdate=" + str(end_date) + "&symbol=" + symbol,
                      data_format='pandas')
    
    if df is None:
        print(df, msg)
    
    df.index = df['price_date'].astype(np.integer)
    df.sort_index(inplace=True)
    return df

def get_index_map(api, symbols, price_date):
    
    symbollist = symbols.split(",")
    
    result = {}
    
    for symbol in symbollist:
    
        df, msg = api.query(
                    view="lb.mfTrackingIndex",
                    fields="",
                    filter="symbol=" + symbol + "&trade_date=" + str(price_date),
                    data_format='pandas')
        
        if df is not None and len(df) > 0:
            result[symbol] = df.loc[0]['index_code']
        
        time.sleep(0.01)
        
    return result


def get_index_daily(api, symbol, start, end):
    
    df, msg = api.daily(
                    symbol=symbol,
                    fields="",
                    start_date=start,
                    end_date=end,
                    data_format='pandas')

    if df is None:
        print(df, msg)
    
    df.index = df['trade_date']
    df.sort_index(inplace=True)
    
    return df

def cal_active_return(api, symbol, bench, start, end):

    df_nav = get_fundnav(api, symbol, start, end)
    df_idx = get_index_daily(api, bench, start, end)
    
    if df_idx.empty or df_nav.empty:
        return None, None, None 
    
    strategy_value = df_nav['nav_adjusted']
    bench_value = df_idx['close']
    
    market_values = pd.concat([strategy_value, bench_value], axis=1).fillna(method='ffill')
    market_values.columns = ['strat', 'bench']
    
    df_returns = market_values.pct_change(periods=1).fillna(0.0)
    
    df_returns = df_returns.join((df_returns.loc[:, ['strat', 'bench']] + 1.0).cumprod(), rsuffix='_cum')

    df_returns.loc[:, 'active_cum'] = df_returns['strat_cum'] - df_returns['bench_cum'] + 1
    df_returns.loc[:, 'active'] = df_returns['active_cum'].pct_change(1).fillna(0.0)

    start = pd.to_datetime(start, format="%Y%m%d")
    end = pd.to_datetime(end, format="%Y%m%d")
    years = (end - start).days / 365.0
    
    active_cum = df_returns['active_cum'].values
    max_dd_start = np.argmax(np.maximum.accumulate(active_cum) - active_cum)  # end of the period
    max_dd_end = np.argmax(active_cum[:max_dd_start])  # start of period
    max_dd = (active_cum[max_dd_end] - active_cum[max_dd_start]) / active_cum[max_dd_start]

    performance_metrics = dict()

    performance_metrics['Annual Return (%)'] =\
        100 * (np.power(df_returns.loc[:, 'active_cum'].values[-1], 1. / years) - 1)
    performance_metrics['Annual Volatility (%)'] =\
        100 * (df_returns.loc[:, 'active'].std() * np.sqrt(242))
    performance_metrics['Sharpe Ratio'] = (performance_metrics['Annual Return (%)']
                                                / performance_metrics['Annual Volatility (%)'])
    
    risk_metrics = dict()
    
    risk_metrics['Beta'] = np.corrcoef(df_returns.loc[:, 'bench'], df_returns.loc[:, 'strat'])[0, 1]
    risk_metrics['Maximum Drawdown (%)'] = max_dd * 100
    risk_metrics['Maximum Drawdown start'] = df_returns.index[max_dd_start]
    risk_metrics['Maximum Drawdown end'] = df_returns.index[max_dd_end]
    
    return performance_metrics, risk_metrics, df_returns

'''
    df_return = (df_nav_curr['nav_adjusted'] / df_nav_start['nav_adjusted']) - 1
    df_return = df_return.sort_values().dropna()
    
    fig, ax1 = plt.subplots(1, 1, figsize=(10, 4))
    ax1.hist(df_return, bins=200)
    ax1.set(xlabel='Return', ylabel='Number', title='Return List')
    fig.show()
    
    # 统计指标
    print(df_return.describe())
    
    # 中位数
    np.median(df_return)
    
    print(len(df_return))
    print(df_return.tail(20))
        
    pass

'''


In [ ]:
api = DataApi('tcp://data.tushare.org:8910')

username = "phone"
password = "token"

df, msg = api.login(username, password)
print(df, msg)

df = get_fundlist(api, u'股票型', u'增强指数型')

symbols = ",".join(df['symbol'])

start_date = 20161230
curr_date  = 20171215

index_map = get_index_map(api, symbols, start_date)

print(index_map)

indicators = list()

for (symbol, index) in index_map.items():

    performance_metrics, risk_metrics, df_returns = cal_active_return(api, symbol, index, start_date, curr_date)

    if performance_metrics is None:
        continue

    indicators.append((symbol, 
                       index, 
                       performance_metrics['Annual Return (%)'],
                       performance_metrics['Annual Volatility (%)'],
                       performance_metrics['Sharpe Ratio'],
                       df_returns['strat_cum'].iat[-1],
                       df_returns['bench_cum'].iat[-1],
                       risk_metrics['Beta'],
                       risk_metrics['Maximum Drawdown (%)'],
                       risk_metrics['Maximum Drawdown start'],
                       risk_metrics['Maximum Drawdown end'])
                    )

labels = ['symbol', 'index', 'AnnualReturn', 'AnnualVolatility', 'SharpeRatio', 'StratCumReturn', 'BenchCumReturn', 'Beta', 'MaximumDrawdown', 'MaximumDrawdownStart', 'MaximumDrawdownEnd']

df = pd.DataFrame.from_records(indicators, columns=labels)

df = df.sort_values('SharpeRatio')

df


In [ ]:
df.sort_values('AnnualReturn')

In [ ]:
df.describe()